In [5]:
import plotly.express as px
from mainnet_launch.constants import *
from mainnet_launch.pages.rebalance_events.rebalance_events import fetch_rebalance_events_df
from mainnet_launch.database.schema.postgres_operations import get_full_table_as_df
from mainnet_launch.database.schema.full import *


autopool = AUTO_USD
df = fetch_rebalance_events_df(autopool)
autopool_state_df = get_full_table_as_df(
    AutopoolStates, where_clause=AutopoolStates.autopool_vault_address == autopool.autopool_eth_addr
)[["block", "total_nav"]]

df2 = pd.merge(df, autopool_state_df, on="block", how="left")
df2.index = df.index

2025-06-03 16:14:36.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:14:36.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:14:36.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:14:36.477 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:14:36.478 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:14:36.478 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:14:36.479 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:14:36.479 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2025-06-03 16:14:36,555 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-03 16:14:36,555 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-06-03 16:14:36,556 INFO sqlalchemy.engine.Engine [cached since 38.85s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x1273fa500>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-06-03 16:14:36,556 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM destinations
            WHERE destinations.chain_id = 1
        
2025-06-03 16:14:36,55

In [6]:
px.bar(df2["spot_slippage_bps"])

In [13]:
df2["swap_cost_in_bps_of_value_out"] = df2["spot_slippage_bps"]
df2["swap_cost_in_bps_of_aum"] = 10_000 * df2["spot_swap_cost"] / df2["total_nav"]

px.bar(df2["swap_cost_in_bps_of_value_out"])

In [17]:
px.bar(df2.resample("1d")["swap_cost_in_bps_of_aum"].mean())

In [14]:
px.bar(df2["swap_cost_in_bps_of_aum"])

In [ ]:
df["adjusted_spot_swap_cost"] = df["spot_swap_cost"] - df["spot_value_in_solver_change"]
df["adjusted_slippage"] = 10_000 * df["adjusted_spot_swap_cost"] / df["spot_value_out"]

px.line(df["adjusted_slippage"], title="slippage_bps_over_time")

In [ ]:
px.bar(df.groupby("move_name")["adjusted_slippage"].mean())

In [ ]:
df.groupby("move_name")["adjusted_slippage"].mean().sort_values()

In [ ]:
px.bar(df.resample("7d")["adjusted_slippage"].mean())

In [ ]:
from mainnet_launch.pages.key_metrics.key_metrics import fetch_nav_per_share_and_total_nav

nav_per_share_df = fetch_nav_per_share_and_total_nav(AUTO_USD)

In [ ]:
nav_per_share_df

In [ ]:
df.columns

In [ ]:
px.bar(df.resample("1d")["adjusted_spot_swap_cost"].sum(), title="total adjusted daily swap costs")

In [ ]:
nav_per_share_df["daily_adjusted_swap_costs"] = df.resample("1d")["adjusted_spot_swap_cost"].sum()
nav_per_share_df["daily_slippage_as_a_chunk_of_nav"] = (
    10_000 * nav_per_share_df["daily_adjusted_swap_costs"] / nav_per_share_df["NAV"]
)
px.bar(nav_per_share_df["daily_slippage_as_a_chunk_of_nav"])

In [ ]:
nav_per_share_df["7_day_rolling_sum_adjusted_swap_costs"] = (
    df.resample("1d")["adjusted_spot_swap_cost"].sum().rolling(7).sum()
)
nav_per_share_df["7_day_rolling_swap_cost_slippage"] = (
    10_000 * nav_per_share_df["7_day_rolling_sum_adjusted_swap_costs"] / nav_per_share_df["NAV"]
)
px.bar(nav_per_share_df["7_day_rolling_swap_cost_slippage"])

In [ ]:
nav_per_share_df[["NAV"]]

In [ ]:
df.columns

In [ ]:
df.to_csv("raw_auto_usd_adjusted_slippage.csv")